In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from toy_transformer import Transformer
import torch

2.0.1


In [3]:
import pandas as pd

In [4]:
toydata = pd.read_csv('medium_articles/eng_de.csv', header=None, names=['en', 'de'])
toydata

,en,de
0,i want to eat bread,ich mochten brot essen
1,i want to eat apple,ich mochten apfel essen
2,i want to drink water,ich mochten wasser trinken
3,i want to drink beer,ich mochten bier trinken
4,i want to read book,ich mochten buch lesen
5,i want to read newspaper,ich mochten zeitung lesen
6,i can eat bread,ich konnen brot essen
7,i can eat apple,ich konnen apfel essen
8,i can drink water,ich konnen wasser trinken
9,i can drink beer,ich konnen bier trinken


In [5]:
def gen_ufunc(nin, nout):
    return lambda f: np.frompyfunc(f, nin, nout)

In [6]:
def reduce(f, arr, dims):
    r = arr
    for dim in dims:
        r = f.reduce(r, dim, keepdims=True)
    return r.squeeze()


In [7]:
import numpy as np

In [8]:

def gen_collect_func(f):
    @gen_ufunc(1, 1)
    def collect_cell(cell):
        return f(cell)
    return collect_cell
    
def collect_cell(df, f):
    res = reduce(gen_collect_func(f), df.to_numpy(), [0, 1])
    return res
    

In [9]:
vocab = toydata.applymap(lambda x: x.split()).apply(np.sum).apply(set)

In [10]:
src_vocab_size = len(vocab['en'])
target_vocab_size = len(vocab['de'])
num_layers = 6
seq_length= 12


In [11]:
model = Transformer(embed_dim=512, src_vocab_size=src_vocab_size, 
                    target_vocab_size=target_vocab_size, seq_length=seq_length,
                    num_layers=num_layers, expansion_factor=4, n_heads=8)
model

Transformer(
  (encoder): TransformerEncoder(
    (embedding_layer): Embedding(
      (embed): Embedding(14, 512)
    )
    (positional_encoder): PositionalEmbedding()
    (layers): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadAttention(
          (query_matrix): Linear(in_features=64, out_features=64, bias=False)
          (key_matrix): Linear(in_features=64, out_features=64, bias=False)
          (value_matrix): Linear(in_features=64, out_features=64, bias=False)
          (out): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (feed_forward): Sequential(
          (0): Linear(in_features=512, out_features=2048, bias=True)
          (1): ReLU()
          (2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): 

In [12]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

@register_line_magic
def print_var(var):
    return print(var, eval(var))

@register_cell_magic
def skip(line, cell):
    "my cell magic"
    return


In [13]:
%%skip
pip install -U torchdata
pip install -U spacy
python -m spacy download en_core_web_sm
python -m spacy download de_core_news_sm

In [14]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List

In [15]:
# We need to modify the URLs for the dataset since the links to the original dataset are broken
# Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

TGT_LANGUAGE = 'de'
SRC_LANGUAGE = 'en'

# Place-holders
token_transform = {}
vocab_transform = {}

In [16]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


In [17]:
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for i, data_sample in data_iter:
        yield token_transform[language](data_sample[language])

# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    # train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_iter = toydata.iterrows()
    
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)


In [18]:
%%skip
pip install portalocker>=2.0.0

In [19]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [20]:
import torch.nn as nn
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')

In [21]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

# transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
#                                  NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
model = Transformer(embed_dim=512, src_vocab_size=SRC_VOCAB_SIZE, 
                    target_vocab_size=TGT_VOCAB_SIZE, seq_length=seq_length,
                    num_layers=3, expansion_factor=4, n_heads=8)
transformer = model

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [22]:
def pad_transform(tokens):
    return tokens + [PAD_IDX] * (seq_length - len(tokens))

In [23]:
token_transform

{'en': functools.partial(<function _spacy_tokenize at 0x000002A2767F27A0>, spacy=<spacy.lang.de.German object at 0x000002A2070C6F80>),
 'de': functools.partial(<function _spacy_tokenize at 0x000002A2767F27A0>, spacy=<spacy.lang.en.English object at 0x000002A20729C130>)}

In [24]:
en_trans = token_transform['en']
en_voc_trans = vocab_transform['en']

In [25]:
en_trans(toydata['en'].iloc[0])

['i', 'want', 'to', 'eat', 'bread']

In [26]:
en_voc_trans(en_trans(toydata['en'].iloc[0]))

[4, 11, 10, 8, 15]

In [27]:
pad_transform(en_voc_trans(en_trans(toydata['en'].iloc[0])))

[4, 11, 10, 8, 15, 1, 1, 1, 1, 1, 1, 1]

In [28]:
from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

# ``src`` and ``tgt`` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               pad_transform,
                                               tensor_transform) # Add BOS/EOS and create tensor


def pad_first_to_len(seqs, l):
    seqs[0] += [0] * (l - len(seqs[0]))
    
# function to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    # pad_first_to_len(src_batch, seq_length)
    # pad_first_to_len(tgt_batch, seq_length)
    
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [29]:
def ToyData(split='train'):
    if split == 'train':
        return toydata.to_numpy()[:20]
    elif split == 'valid':
        return toydata.to_numpy()[20:30]
    else:
        return toydata.to_numpy()[30:]

In [35]:
from torch.utils.data import DataLoader

def train_epoch(model, optimizer):
    model.train()
    losses = 0
    # train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_iter = ToyData(split='train')
    train_dataloader = DataLoader(train_iter, batch_size=10, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        # logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        # logits = model(src, tgt_input)
        logits = model(src, tgt)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        # loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))

In [38]:
def evaluate(model):
    model.eval()
    losses = 0

    # val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_iter = ToyData(split='valid')
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        # logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        # logits = model(src, tgt_input)
        logits = model(src, tgt)

        tgt_out = tgt[1:, :]
        # loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [40]:
from timeit import default_timer as timer
NUM_EPOCHS = 18

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

2.0.1
Epoch: 1, Train loss: 2.081, Val loss: 2.334, Epoch time = 7.127s
Epoch: 2, Train loss: 2.073, Val loss: 2.332, Epoch time = 6.874s
Epoch: 3, Train loss: 2.065, Val loss: 2.330, Epoch time = 6.817s
Epoch: 4, Train loss: 2.063, Val loss: 2.327, Epoch time = 7.061s
Epoch: 5, Train loss: 2.064, Val loss: 2.325, Epoch time = 7.026s
Epoch: 6, Train loss: 2.062, Val loss: 2.323, Epoch time = 7.078s
Epoch: 7, Train loss: 2.062, Val loss: 2.321, Epoch time = 7.120s
Epoch: 8, Train loss: 2.061, Val loss: 2.319, Epoch time = 6.997s
Epoch: 9, Train loss: 2.059, Val loss: 2.319, Epoch time = 6.950s
Epoch: 10, Train loss: 2.060, Val loss: 2.319, Epoch time = 7.159s
Epoch: 11, Train loss: 2.059, Val loss: 2.319, Epoch time = 7.096s
Epoch: 12, Train loss: 2.052, Val loss: 2.318, Epoch time = 7.118s
Epoch: 13, Train loss: 2.050, Val loss: 2.318, Epoch time = 7.255s
Epoch: 14, Train loss: 2.053, Val loss: 2.321, Epoch time = 7.014s
Epoch: 15, Train loss: 2.043, Val loss: 2.325, Epoch time = 7.141

In [56]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    # memory = model.encode(src, src_mask)
    memory = model.encode(src)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        # out = model.decode(ys, memory, tgt_mask)
        out = model.decode(ys, memory)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt = torch.ones(1, 1).fill_(BOS_IDX).type(torch.long).to(DEVICE)
    # tgt_tokens = greedy_decode(
    #     model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    model.decode(src, tgt)
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

2.0.1


In [ ]:
import pickle
pickle.dump(transformer, open('transformer.pickle', 'wb'))

In [57]:
translate(transformer, toydata.iloc[0]['en'])

RuntimeError: shape '[14, 1, 8, 64]' is invalid for input of size 512